In [ ]:
import mlflow
from bs4.diagnose import profile
from dotenv import load_dotenv
from databricks.connect import DatabricksSession

spark = DatabricksSession.builder.getOrCreate()
load_dotenv('../.env')

local = True
if local:
    # local mlflow setup
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_registry_uri("http://localhost:5000")
else:
    mlflow.set_tracking_uri("databricks")
    mlflow.set_registry_uri("databricks-uc")

mlflow.set_experiment("langgraph_workflow_local_dev")
mlflow.langchain.autolog()

In [ ]:
from unitycatalog.ai.core.base import set_uc_function_client
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()
set_uc_function_client(client)

CATALOG = 'fins_genai'
SCHEMA = 'agents'

# Create Sequence of Steps

In [ ]:
from typing import Any, Generator, Optional, Sequence, Union, Annotated
from typing_extensions import TypedDict
from langchain_core.language_models import LanguageModelLike
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.tools import BaseTool
from langgraph.graph import START, END, StateGraph
from langgraph.graph.graph import CompiledGraph
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt.tool_node import ToolNode
from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)

In [ ]:
class State(TypedDict):
    model: Annotated[str, "model name"]
    plan: Annotated[str, "document outline"]
    draft: Annotated[str, "document draft"]
    final_doc: Annotated[str, "final document"]


